# Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load Data

In [3]:
# specify path to the file here
path = '/content/BNPs table.xlsx'
data = pd.read_excel('/content/BNPs table.xlsx')

# check the format of the data
data.head()

,BMP,Cost,NitrateLoadReduction
0,Nutrient Management,11.6,13
1,Conservation Tillage,10.0,68
2,Cover Crops,105.1,84
3,Riparian Buffer,23.4,80


# ANP connection efficiencies, populating Supermatrix

In [4]:
# Number of alternatives (BNPs) are read using data.shape[0]
SupMat = np.zeros((data.shape[0],data.shape[0]))
for row in range(data.shape[0]):  # iternating over rows
  for col in range(data.shape[1]+1):  # iterating over cols
   # using the formula from the paper to calculate network efficiencies
   if row==col:
    SupMat[row,col] = data.NitrateLoadReduction[row]/data.Cost[row]
   else:
    SupMat[row,col] = (data.NitrateLoadReduction[row]+data.NitrateLoadReduction[col])/(data.Cost[row]+data.Cost[col])

print(SupMat)


[[1.12068966 3.75       0.83119109 2.65714286]
 [3.75       6.8        1.32059079 4.43113772]
 [0.83119109 1.32059079 0.79923882 1.27626459]
 [2.65714286 4.43113772 1.27626459 3.41880342]]


# Column normalized supermatrix

$$ fun = \frac{A_{ii}}{A_i}$$

In [12]:
J =  np.ones_like(SupMat)
np.matmul(SupMat,J)

NormSupMat = np.divide(SupMat,np.matmul(SupMat,J))
print(NormSupMat)

[[0.13406945 0.44861699 0.09943638 0.31787718]
 [0.23003696 0.41713368 0.08100925 0.27182012]
 [0.19662526 0.31239689 0.18906669 0.30191116]
 [0.22549981 0.37605081 0.10831086 0.29013853]]


# Stabilizing the Supermatrix to compute LimitMatrix

In [ ]:
LimMat = np.linalg.matrix_power(NormSupMat,200)
print(LimMat)

[[0.20552591 0.40081566 0.10393758 0.28972085]
 [0.20552591 0.40081566 0.10393758 0.28972085]
 [0.20552591 0.40081566 0.10393758 0.28972085]
 [0.20552591 0.40081566 0.10393758 0.28972085]]


# Saving efficiencies in the datafile

In [ ]:
data.insert(data.shape[1],"Eff", LimMat[0,:], True)

# Selecting best $n$ BMPs

In [ ]:
n = int(input('Enter value of n:'))

Enter value of n:2


In [ ]:
df1 = data.sort_values(by=['Eff'],ascending=False)

In [ ]:
df1.BMP[0:n]

1    Conservation Tillage
3         Riparian Buffer
Name: BMP, dtype: object

# Selecting 2nd best $n$ BMPs

In [ ]:
df2 = df1.reset_index(drop=True)

In [ ]:
df2.Eff[n-1] = 0

<ipython-input-230-451de3bc1426>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.Eff[n-1] = 0


In [ ]:
df2 = df2.sort_values(by=['Eff'],ascending=False)
df2.BMP[0:n]

0    Conservation Tillage
2     Nutrient Management
Name: BMP, dtype: object

# Selecting 3rd best $n$ BMPs

In [ ]:
df3 = df1.reset_index(drop=True)
df3

,BMP,Cost,NitrateLoadReduction,Eff
0,Conservation Tillage,10.0,68,0.400816
1,Riparian Buffer,23.4,80,0.289721
2,Nutrient Management,11.6,13,0.205526
3,Cover Crops,105.1,84,0.103938


In [ ]:
df3.Eff[n-2] = 0
df3 = df3.sort_values(by=['Eff'],ascending=False)
df3.BMP[0:n]

<ipython-input-233-403621ec5e12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.Eff[n-2] = 0


1        Riparian Buffer
2    Nutrient Management
Name: BMP, dtype: object